## **Customer Supper Multi Agent using build in `tools`**

In [16]:
import os
from dotenv import load_dotenv
load_dotenv()
from crewai import LLM

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

llm = LLM(model="groq/meta-llama/llama-4-scout-17b-16e-instruct")

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Crew, Task

## **Support Agent**

In [4]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
    llm=llm
    
)

- By not setting allow_delegation=False, allow_delegation takes its default value of being True.
- This means the agent can delegate its work to another agent which is better suited to do a particular task.

In [6]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
    llm=llm
)

## **Incorporate tools and Memory**

In [7]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## **Defining the tools**

In [17]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Way to use the tools
- tools use as the `RAG` type task like we can specify the specific link to extract or semantic search on the link

In [18]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [10]:
docs_scrape_tool

ScrapeWebsiteTool(name='Read website content', description="Tool Name: Read website content\nTool Arguments: {}\nTool Description: A tool that can be used to read https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/'s content.", env_vars=[], args_schema=<class 'crewai_tools.tools.scrape_website_tool.scrape_website_tool.FixedScrapeWebsiteToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x000001EB082E56C0>, result_as_answer=False, max_usage_count=None, current_usage_count=0, website_url='https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/', cookies=None, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 'Accept-Language': 'en-US,en;q=0.9', 'Referer': 'https://www.google.com/', 'Connection': 

In [13]:
docs_scrape_tool.run()

Using Tool: Read website content


'Introduction - CrewAI CrewAI home page English Search... ⌘ K Ask AI Start Cloud Trial crewAIInc / crewAI crewAIInc / crewAI Search... Navigation Get Started Introduction Documentation Enterprise API Reference Examples Website Forum Crew GPT Get Help Releases Get Started Introduction Installation Quickstart Guides Strategy Agents Crews Flows Advanced Core Concepts Agents Tasks Crews Flows Knowledge LLMs Processes Collaboration Training Memory Reasoning Planning Testing CLI Tools Event Listeners MCP Integration MCP Servers as Tools in CrewAI Stdio Transport SSE Transport Streamable HTTP Transport Connecting to Multiple MCP Servers MCP Security Considerations Tools Tools Overview File & Document Web Scraping & Browsing Search & Research Database & Data AI & Machine Learning Cloud & Storage Automation & Integration Observability Overview AgentOps Integration Arize Phoenix Langfuse Integration Langtrace Integration Maxim Integration MLflow Integration OpenLIT Integration Opik Integration P

## **Creating Tasks**

In [22]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [23]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


## **Creating Crew**

In [32]:
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    verbose=True,
    
)

In [33]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 80cd9a27-21c3-439f-b001-9713bd744de4                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Task: DeepLearningAI just reached out with a super important ask:                                              │
│  I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you   │
│  provide guidance?                                                                                              │
│                                                                                                                 │
│  Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide     │
│  the best support possible.You must strive to provide a complete and accurate response to the customer's        │
│  inquiry.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Thought: Thought: I need to find information on how to add memory to a Crew and kick it off for                │
│  DeepLearningAI. The best place to start is by reading the documentation on creating a Crew and kicking it off  │
│  from CrewAI's website.                                                                                         │
│                                                                                                                 │
│  Using Tool: Read website content                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"url\": \"https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/\"}"                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Introduction - CrewAI CrewAI home page English Search... ⌘ K Ask AI Start Cloud Trial crewAIInc / crewAI       │
│  crewAIInc / crewAI Search... Navigation Get Started Introduction Documentation Enterprise API Reference        │
│  Examples Website Forum Crew GPT Get Help Releases Get Started Introduction Installation Quickstart Guides      │
│  Strategy Agents Crews Flows Advanced Core Concepts Agents Tasks Crews Flows Knowledge LLMs Processes           │
│  Collaboration Training Memory Reasoning Planning Testing CLI Tools Event Listeners MCP Integration MCP         │
│  Servers as Tools in CrewAI Stdio Transport SSE Transport Streamable HTTP Transport Connecting to Multiple MCP  │
│  Servers MCP Security Considerations Tools Tools Overview File & Document Web Scraping & Browsing Search &      │
│  Research Database & Data AI & Machine Learning Cloud & Storage Automation & Integration Observability          │
│  Overview AgentOps Integration Arize Phoenix Langfuse Integration Langtrace Integration Maxim Integration       │
│  MLflow Integration OpenLIT Integration Opik Integration Patronus AI Evaluation Portkey Integration Weave       │
│  Integration Learn Overview Strategic LLM Selection Guide Conditional Tasks Coding Agents Create Custom Tools   │
│  Custom LLM Implementation Custom Manager Agent Customize Agents Image Generation with DALL-E Force Tool        │
│  Output as Result Hierarchical Process Human Input on Execution Kickoff Crew Asynchronously Kickoff Crew for    │
│  Each Connect to any LLM Using Multimodal Agents Replay Tasks from Latest Crew Kickoff Sequential Processes     │
│  Using Annotations in crew.py Telemetry Telemetry Get Started Introduction Copy page Build AI agent teams that  │
│  work together to tackle complex tasks ​ What is CrewAI?                                                         │
│  CrewAI is a lean, lightning-fast Python framework built entirely from scratch—completely independent of        │
│  LangChain or other agent frameworks.                                                                           │
│  CrewAI empowers developers with both high-level simplicity and precise low-level control, ideal for creating   │
│  autonomous AI agents tailored to any scenario:                                                                 │
│  CrewAI Crews : Optimize for autonomy and collaborative inte...                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for     │
│  model `meta-llama/llama-4-scout-17b-16e-instruct` in organization `org_01jm7jy752frk82tnecnr6svyr` service     │
│  tier `on_demand` on tokens per minute (TPM): Limit 30000, Used 24758, Requested 7063. Please try again in      │
│  3.641s. Need more tokens? Upgrade to Dev Tier today at                                                         │
│  https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


 An unknown error occurred. Please check the details below.



╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 693c7834-9ce7-416d-a46f-cd026adf41fe                                                                     │
│  Agent: Senior Support Representative                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 80cd9a27-21c3-439f-b001-9713bd744de4                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `meta-llama/llama-4-scout-17b-16e-instruct` in organization `org_01jm7jy752frk82tnecnr6svyr` service tier `on_demand` on tokens per minute (TPM): Limit 30000, Used 24758, Requested 7063. Please try again in 3.641s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}
